In [80]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
sys.path.append("../")
import torch
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from molecule_optimizer.externals.fast_jtnn.datautils import MolTreeFolder, PairTreeFolder, MolTreeDataset,  SemiMolTreeFolder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [82]:
from dig.ggraph.dataset import ZINC250k, ZINC800
from dig.ggraph.evaluation import RandGenEvaluator

In [83]:
path_adj = ''
dataset = ZINC800(one_shot=False, root='data')

In [84]:
zinc_800_jt = torch.load("data/zinc_800_jt/processed/data.pt")

In [85]:
smiles = zinc_800_jt[-1]
labels = zinc_800_jt[0]

In [86]:
# Define model
jtvae = SemiJTVAEGeneratorPredictor(smiles)
config = {"hidden_size": 420, "latent_size": 56, "depthT": 20, "depthG": 3, "label_size": 1, "label_mean": 0, "label_var": 1}
jtvae.get_model('rand_gen', config)

/home/tsa87/anaconda3/envs/anbnet/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
preprocessed, labels = jtvae.preprocess(smiles, labels)

100%|██████████| 800/800 [07:49<00:00,  1.70it/s]


AttributeError: 'bool' object has no attribute 'nonzero'

In [45]:
loader = SemiMolTreeFolder(preprocessed, labels, jtvae.vocab, 32, num_workers=0)
load_epoch = 0
lr = 1e-3
anneal_rate = 0.9
clip_norm = 50.0
num_epochs = 1
total_step = 0  # TODO args.load_epoch
beta = 0.0  # TODO args.beta
max_beta = 1.0
save_iter = 5000
step_beta = 0.002
anneal_iter = 40000
print_iter = 50
kl_anneal_iter = 2000
jtvae.train_rand_gen(loader, load_epoch, lr, anneal_rate, clip_norm, num_epochs, beta, max_beta, step_beta, anneal_iter, kl_anneal_iter, print_iter, save_iter)

Model #Params: 4260K


In [46]:
samples = list(map(Chem.MolFromSmiles, jtvae.run_rand_gen(30)))

In [47]:
res_dict = {'mols': samples, 'train_smiles': smiles}
evaluator = Rand_Gen_Evaluator()
results = evaluator.eval(res_dict)
print(results)

Valid Ratio: 30/30 = 100.00%
Unique Ratio: 17/30 = 56.67%
Novel Ratio: 30/30 = 100.00%
{'valid_ratio': 100.0, 'unique_ratio': 56.666666666666664, 'novel_ratio': 100.0}
